In [ ]:
# Importando las librearias necesarias para trabajr con Langchain
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [ ]:
# Una manera limpia de manejar las API Keys es usando la libreria .env
from dotenv import load_dotenv
import os


In [ ]:
# Leyendo nuestro limpio dataset a un pandas dataframe llamado books
import pandas as pd
books = pd.read_csv('books_cleaned.csv')
books.head()

In [ ]:
# Ahora es momento de construir nuestro vector search
# el metodo TextLoader de Langchain no trabaja con Pandas Dataframes, por lo que se tiene que guardar
# los tag descriptions solo en un archivo de texto que estén separados por saltos de lineas, sin index y sin header
books['tagged_description'].to_csv('tagged_description.txt',sep="\n", index=False, header=False)

In [ ]:
# Con las lineas de abajo se busca cargar el texto de nuestro archivo de texto y dividirlo en documentos individuales
# 1. Cargamos el texto de nuestro archivo de texto
# 2. Dividimos el texto en documentos individuales: Con chunk_size=0 y chunk_overlap=0, estamos diciendo que cada documento es una linea
# 3. Cada documento es un tag description

# Carga el texto de nuestro archivo de texto a un objeto de tipo Document
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load() 
#raw_documents = TextLoader("tagged_description.txt").load()  # Cargamos el texto de nuestro archivo de texto
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
# Esta linea de codigo con chunk_size=0 indica que cada linea es un documento, y con chunk_overlap=0 indica 
# que no hay overlap entre documentos es decir que no hay palabras que se repitan en dos documentos
documents = text_splitter.split_documents(raw_documents)  # Con esta linea de codigo separamos el texto en documentos individuales
# Explciando lo que hacen las lineas de arriba
# 1. Cargamos el texto de nuestro archivo de texto
# 2. Dividimos el texto en documentos individuales: Con chunk_size=0 y chunk_overlap=0, estamos diciendo que cada documento es una linea
# 3. Cada documento es un tag description


In [ ]:
documents[0] # Mostramos el primer documento


In [ ]:
# Ahora vamos a crear los document embeddings y guardarlos en un vector database
# Usamos el metodo Chroma para crear los embeddings de los documentos y los asiganmos a la variable db_books
# Chroma es un metodo de Langchain que permite crear embeddings de documentos
# Un embedding es una representacion numerica de un documento, que permite buscar documentos similares entre si
# En este caso se esta utilizando el modelo de OpenAI llamado openai-gpt
# ************** Teoria de embeddings **************
''' Un embedding es una representación numérica (un vector) que captura el contenido de un texto. 
Permite comparar documentos entre sí midiendo la similitud entre sus vectores.

Ejemplo práctico: Supongamos que tienes dos frases:

“El gato come pescado.”
“El gato come atún.”
Al convertirlas en embeddings, obtienes dos vectores (por ejemplo, [0.1, 0.4, 0.8, …] y [0.1, 0.5, 0.7, …]).
Si mides la distancia (p. ej. coseno) entre ambos vectores y ves que es pequeña, concluyes que las oraciones 
son similares en significado. Así, un motor de búsqueda podría encontrar documentos con frases parecidas 
gracias a esos vectores.
'''
# 1. Creamos los embeddings de los documentos
# 2. Los embeddings son guardados en un vector database llamado db_books
# 3. Chroma es un metodo de Langchain que permite crear embeddings de documentos
# Los 2 parametros que recibe Chroma son los documentos y el modelo que se va a utilizar para crear los embeddings
# En este caso se esta utilizando el modelo de OpenAI llamado openai-gpt
db_books = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())




In [ ]:
# Para nuestra primera query busquemos algo especifico
query = "A book to teach childreb about nature"
# Y para obtener las recomendaciones vamos a tomar nuestro vector database y hacer una query
docs = db_books.similarity_search(query, k=2)
# La linea anterior sirve para hacer una query a nuestro vector database y obtener las recomendaciones, los parametros que recibe son
# 1. La query que se va a hacer
# 2. El numero de recomendaciones que se van a obtener
# En este caso se estan obteniendo 2 recomendaciones

docs

El resultado que nos da esta bien, porque nos da las descripciones relacionadas a lo que pedimos; pero lo que en realidad tambien buscamos es el Titulo del libro, por lo que necesitamos de algun modo usar estas recomendaciones para filtrar el data frame que contiene todos los libros.
Y es ahi donde los ISBN se van a usar.

In [ ]:
# Vamos a crear un pandas data frame y en este caso vamos a obtener el primer documento
books[ books["isbn13"] == int(docs[0].page_content.split()[0].strip()) ] # Mostramos el primer documento
# La linea anterior sirve para filtrar el dataframe y obtener el libro que corresponde a la recomendacion
# En este caso se esta filtrando el dataframe por el isbn13 del libro que corresponde a la recomendacion
